# 1.1 First NGSolve example

Let us solve the Poisson problem of finding $u$ satisfying 

$$
\begin{aligned}
-\Delta u & = f && \text { in  the unit square},
\\
u & = 0 && \text{ on the bottom and right parts of the boundary},
\\
\frac{\partial u }{\partial n } & = 0 
&& \text{ on the remaining  boundary parts}.
\end{aligned}
$$

## Quick steps to solution:

#### 1. Import NGSolve and Netgen Python modules:

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

#### 2. Generate an unstructured mesh

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
mesh.nv, mesh.ne   # number of vertices & elements 

(137, 232)

Here we prescribed a maximal mesh-size of 0.2 using the `maxh` flag. 

In [3]:
Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

#### 3. Declare a finite element space:

In [6]:
fes = H1(mesh, order=2, dirichlet="bottom|right")
fes.ndof  # number of unknowns in this space

505

Python's help system displays further documentation.

In [7]:
help(fes)

Help on H1 in module ngsolve.comp object:

class H1(FESpace)
 |  An H1-conforming finite element space.
 |
 |  The H1 finite element space consists of continuous and
 |  element-wise polynomial functions. It uses a hierarchical (=modal)
 |  basis built from integrated Legendre polynomials on tensor-product elements,
 |  and Jaboci polynomials on simplicial elements.
 |
 |  Boundary values are well defined. The function can be used directly on the
 |  boundary, using the trace operator is optional.
 |
 |  The H1 space supports variable order, which can be set individually for edges,
 |  faces and cells.
 |
 |  Internal degrees of freedom are declared as local dofs and are eliminated
 |  if static condensation is on.
 |
 |  The wirebasket consists of all vertex dofs. Optionally, one can include the
 |  first (the quadratic bubble) edge basis function, or all edge basis functions
 |  into the wirebasket.
 |
 |  Keyword arguments can be:
 |
 |  order: int = 1
 |    order of finite element 

#### 4. Declare test function, trial function, and grid function 

* Test and trial function are symbolic objects - called `ProxyFunctions` -  that help you construct bilinear forms (and have no space to hold solutions). 

* `GridFunctions`, on the other hand, represent functions in the finite element space and contains memory to hold coefficient vectors.

In [8]:
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object
gfu = GridFunction(fes)  # solution 

Alternately, you can get both the trial and test variables at once:

In [9]:
u, v = fes.TnT()

#### 5. Define and assemble linear and bilinear forms:

In [10]:
a = BilinearForm(fes)
a += grad(u)*grad(v)*dx
a.Assemble()

f = LinearForm(fes)
f += x*v*dx
f.Assemble();

Alternately, we can do one-liners: 

In [11]:
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
f = LinearForm(x*v*dx).Assemble()

You can examine the linear system in more detail:

In [12]:
print(f.vec)

 4.16667e-05
 0.00259713
 0.001625
 8.94898e-05
 0.000524248
 0.00108866
 0.0014012
 0.00181062
 0.00224583
 0.00260886
 0.00282313
 0.00290497
 0.00327283
 0.00356735
 0.00356399
 0.0039385
 0.00416898
 0.00420942
 0.00422101
 0.00425523
 0.00422508
 0.00490256
 0.00594646
 0.00322428
 0.00326175
 0.00277011
 0.00217396
 0.00166527
 0.00119941
 0.000753088
 0.000396818
 9.47241e-05
 9.32024e-05
 0.00010905
 0.000118838
 0.0001224
 0.000125679
 0.000139174
 0.000206369
 0.000225384
 0.00165026
 0.00238558
 0.00310606
 0.00408058
 0.00499507
 0.00555962
 0.00556139
 0.00519081
 0.00605906
 0.00579454
 0.00698723
 0.00792567
 0.00806885
 0.00795802
 0.00811952
 0.0082474
 0.00844436
 0.00556319
 0.00698973
 0.00665616
 0.00495605
 0.00376814
 0.00285484
 0.00193673
 0.00106455
 0.000515226
 0.000462838
 0.000610731
 0.000707788
 0.000732141
 0.000736791
 0.000749017
 0.000922628
 0.00226065
 0.00162566
 0.00481935
 0.00580575
 0.00604536
 0.00518734
 0.00361145
 0.00782665
 0.00566048
 0

In [13]:
print(a.mat)

Row 0:   0: 1   4: -0.5   39: -0.5   137: -0.0833333   138: -0.0833333   148: 0.166667
Row 1:   1: 0.852428   12: -0.305555   13: -0.305283   48: -0.24159   139: -0.0201768   140: -0.0200882   141: -0.101806   172: 0.0711027   174: 0.0709686
Row 2:   2: 1   21: -0.5   22: -0.5   142: -0.0833333   143: -0.0833333   197: 0.166667
Row 3:   3: 0.857515   30: -0.316112   31: -0.315989   65: -0.225414   144: -0.0188061   145: -0.018763   146: -0.10535   224: 0.0714914   226: 0.0714278
Row 4:   0: -0.5   4: 1.89834   5: -0.534935   39: -0.163225   40: -0.700175   137: 1.38275e-16   138: 0.0833333   147: -0.0569533   148: -0.143076   149: -0.11636   151: 0.146109   249: 0.0869467
Row 5:   4: -0.534935   5: 1.88484   6: -0.357768   40: -0.184145   41: -0.807996   147: -0.0127762   149: 0.101932   150: -0.0513319   151: -0.17249   152: -0.0775426   154: 0.11096   250: 0.101249
Row 6:   5: -0.357768   6: 1.7516   7: -0.321463   41: -0.415088   42: -0.65728   150: -0.0350003   152: 0.0946282   153

#### 6. Solve the system:

In [14]:
gfu.vec.data = \
    a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

The Dirichlet boundary condition constrains some degrees of freedom. The argument `fes.FreeDofs()` indicates that only the remaining "free" degrees of freedom should participate in the linear solve.

You can examine the coefficient vector of solution if needed:

In [15]:
print(gfu.vec)

       0
       0
       0
 0.0923441
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
 0.0332852
 0.0579003
 0.0751595
 0.0863451
 0.0926982
 0.0954153
 0.0956477
 0.094504
 0.093056
 0.0914712
 0.0888246
 0.0843736
 0.0780626
 0.0698262
 0.0596052
 0.0473732
 0.0331736
 0.0172158
 0.0212313
 0.0187597
 0.0189971
 0.0197547
 0.019801
 0.0184571
 0.015625
 0.0119589
 0.00811742
 0.0124347
 0.0170908
 0.0214351
 0.0243813
 0.0261768
 0.0276516
 0.028612
 0.0285798
 0.0489354
 0.0661216
 0.0805337
 0.0894465
 0.09386
 0.0952318
 0.0947501
 0.0935277
 0.0922606
 0.0903319
 0.0869582
 0.0815965
 0.0742017
 0.0648724
 0.0536035
 0.0400366
 0.033972
 0.0346734
 0.0367028
 0.0353636
 0.0310034
 0.0242502
 0.0173425
 0.0338692
 0.0250323
 0.0408636
 0.0445245
 0.0474761
 0.0504058
 0.0512247
 0.084201
 0.0703716
 0.0906047
 0.0932407
 0.093521
 0.0927214
 0.0919286
 0.0855722
 0.089

You can see the zeros coming from the zero boundary conditions.

## Ways to interact with NGSolve

* A jupyter notebook (like this one) gives you one way to interact with NGSolve. When you have a complex sequence of tasks to perform, the notebook may not be adequate.


* You can write an entire python module in a text editor and call python on the command line. (A script of the above is provided in `poisson.py`.)
    ```
    python3 poisson.py
    ```
  
* If you want the Netgen GUI, then use `netgen` on the command line:
    ```
    netgen poisson.py
    ```
  You can then ask for a python shell from the GUI's menu options (`Solve -> Python shell`).
  